In [1]:
# Importações

import os
from dotenv import load_dotenv
from datetime import datetime

import yfinance as yf

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import Markdown

In [2]:
# Carregando variáveis de ambiente

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
# Criando yahoo finance tool

def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
    return stock


yahoo_finance_tool = Tool(
    name="Yahoo Finance Tool",
    description="Fetch stock prices to {ticket} from the last year about a specific stock of the Yahoo Finance API.",
    func=lambda ticket: fetch_stock_price(ticket),
)

In [4]:
# Importando Openai LLM - GPT

llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=200, api_key=OPENAI_API_KEY)

In [5]:
# Criando agente para analisar o preço de ações

stock_price_analyst = Agent(
    role="Senior Stock Price Analyst",
    goal="Find the {ticket} stock price and analyses trends",
    backstory="""You're a highly experienced in analyzing the price of an specific stock and make predictions about its future price.""",
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[yahoo_finance_tool],
    allow_delegation=False,
)

In [6]:
# Criando tarefa para obter o preço da ação

get_stock_price = Task(
    description="Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
    expected_output=""" Specify the current trend stock price - up, down or sideways.
    eg. stock= 'APPL, price UP' """,
    agent=stock_price_analyst,
)

In [7]:
# Importando search tool

search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [8]:
# Criando agente para analisar notícias de ações

news_analyst = Agent(
    role="Stock News Analyst",
    goal=""" Create a short summary of the market related to the stock {ticket} company. 
    Specify the current trend - up, down or sideways with the news context. 
    For each request stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed. """,
    backstory=""" You-re highly experienced in analyzing the market and news and have tracked assest for more than 10 years.
    You're also master level analysts in the tradicional markets and have deep understanding of human psychology.
    You understand news, theirs titles and information, but you look at those with a health dose of skepticism.
    You consider also the source of the news articles. """,
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[search_tool],
    allow_delegation=False,
)

In [9]:
# Criando tarefa para obter notícias de ações

get_news = Task(
    description=f"""Take the stock and always include BTC to it (if no request).
    Use the search tool to search each one individually.
    The request date is {datetime.now()}.
    Compose the results into a helpfull report. """,
    expected_output=""" A summary of the overall market and one sentence summary for each request asset.
    Include a fear/greed score for each asset based on the news. 
    Use format:
        <STOCK ASSET>
        <SUMARRY BASED ON NEWS>
        <TREND PREDICTION>
        <FEAR/GREED SCORE>
    """,
    agent=news_analyst,
)

In [10]:
# Criando agente para escrever análise de ações

stock_analyst_write = Agent(
    role="Senior Writer",
    goal="""Analyze the trends price and news and write and insighfull compelling and informative 3 paragraph long newsletter based on the stock report and price trend.""",
    backstory=""" You're widely accpeted as athe best stock analyst in the market. 
    You understand complex concepts and create compelling stories and narratives that resonate with wider audiences.
    You understand macro factors and combine multiple theories - eg. cycle theory fundamental analyses.
    You're able to hold multiple opnions when analyzing anything.""",
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    allow_delegation=True,
)

In [11]:
# Criando tarefa para escrever análise de ações

write_analysis = Task(
    description=""" Use the stock price and the stock news report to create an analysis and write the newsletter 
    about the {ticket} company that is brief and highligts the most important points.
    Focus on the sock price trend, news and fear/greed score. What are the near future considerations? 
    Include the previous analysis of stock trend and news summary. """,
    expected_output=""" An eloquent 3 paragraph newsletter formated as markdown in an easy readable manner. It should contain:
        - 3 bullets executive summary.
        - Introduction - set the overerall picture and spike up the interest.
        - main part provides the meat of the analysis including the news summary and feed/greed scores.
        - summary - key facts and concrete future trend prediction - up, down or sideways.
    """,
    agent=stock_analyst_write,
    context=[get_stock_price, get_news],
)

In [12]:
# Criando grupo de IA

crew = Crew(
    agents=[stock_price_analyst, news_analyst, stock_analyst_write],
    tasks=[get_stock_price, get_news, write_analysis],
    verbose=True,
    process=Process.hierarchical,
    full_output=True,
    share_crew=True,
    manager_llm=llm,
    max_iter=15,
)

In [13]:
# Executando a análise

results = crew.kickoff(inputs={"ticket": "AAPL"})

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyze the stock AAPL price history and create a trend analyses of up, down or sideways


> Entering new CrewAgentExecutor chain...
I need to gather information on the stock AAPL price history and create a trend analysis to determine if it's currently up, down, or sideways. I should delegate the task of analyzing the stock price history to the Senior Stock Price Analyst.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior Stock Price Analyst", "task": "Analyze AAPL stock price history and determine current trend (up, down, or sideways)", "context": "Provide detailed analysis on the historical stock prices of AAPL and determine the current trend based on the data."}

> Entering new CrewAgentExecutor chain...
I need to gather historical stock price data for AAPL and analyze it to determine the current trend.

Action: Yahoo Finance Tool
Action Input: {"ticket": "AAPL"}

[*********************100%%**********************]  1 of 1 completed


 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643387   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043762   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825150   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884857   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546371   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996  213.500000  196.000000  209.270004  209.028061   
2024-08-06  205.300003  209.990005  201.070007  207.229996  206.990402   
2024-08-07  206.899994  213.639999  206.389999  209.820007  209.577423   

               Volume  
Date      

C:\Users\richa\AppData\Local\Programs\Python\Python311\Lib\asyncio\events.py:84: UserWarning: Curlm alread closed! quitting from process_data
  self._context.run(self._callback, *self._args)


Thought: I need to gather information on the stock market and the specific asset requested (APPL). I should delegate tasks to my team members to ensure the report is comprehensive and accurate.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior Writer", "task": "Write a detailed summary of the overall stock market performance and trends", "context": "The task involves summarizing the current state of the stock market, including any significant changes or trends that may impact trading."}

> Entering new CrewAgentExecutor chain...


...

C:\Users\richa\AppData\Local\Programs\Python\Python311\Lib\asyncio\events.py:84: UserWarning: Curlm alread closed! quitting from process_data
  self._context.run(self._callback, *self._args)


I need to gather the necessary information in order to delegate the work effectively and ensure the final newsletter analysis is top-notch.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior Stock Price Analyst", "task": "Provide analysis of current APPL stock price trend and previous analysis summary", "context": "APPL stock price is currently sideways. Previous analysis indicated an uncertain trend prediction."}

> Entering new CrewAgentExecutor chain...
I need to use the Yahoo Finance Tool to fetch the APPL stock prices from the last year and analyze the trends to provide an updated analysis to my coworker.

Action: Yahoo Finance Tool
Action Input: {"ticket": "AAPL"}

> Entering new CrewAgentExecutor chain...
I should use the Yahoo Finance Tool to fetch the APPL stock prices from the last year to analyze the trend and make a prediction.

Action: Yahoo Finance Tool
Action Input: {"ticket": "AAPL"}

> Entering new CrewAgentExecutor chain...
I need to analyze the cu

C:\Users\richa\AppData\Local\Programs\Python\Python311\Lib\asyncio\events.py:84: UserWarning: Curlm alread closed! quitting from process_data
  self._context.run(self._callback, *self._args)


Thought: I now can give a great answer

Final Answer:
# AAPL Stock Analysis Newsletter

## Executive Summary:
- AAPL stock price currently remains sideways with uncertain trend prediction.
- Overall market is experiencing volatility with a neutral fear/greed score.
- News related to AAPL is not available at this time.

## Introduction:
Despite market volatility, AAPL continues to hold steady with a sideways stock price trend. Investors are eagerly awaiting the latest news and updates to determine the next move for this tech giant.

## Main Analysis:
Unfortunately, specific news related to AAPL is currently unavailable. However, based on the previous uncertain trend prediction, it is essential for investors to stay informed and monitor the stock closely for any potential changes.

## Future Trend Prediction:
With the market in a volatile state and AAPL stock remaining sideways, it is crucial to keep a close eye on any upcoming developments. The fear/greed score being neutral signifies a

C:\Users\richa\AppData\Local\Programs\Python\Python311\Lib\asyncio\events.py:84: UserWarning: Curlm alread closed! quitting from process_data
  self._context.run(self._callback, *self._args)


In [14]:
Markdown(results["final_output"])

# AAPL Stock Analysis Newsletter

## Executive Summary:
- AAPL stock price currently remains sideways with uncertain trend prediction.
- Overall market is experiencing volatility with a neutral fear/greed score.
- News related to AAPL is not available at this time.

## Introduction:
Despite market volatility, AAPL continues to hold steady with a sideways stock price trend. Investors are eagerly awaiting the latest news and updates to determine the next move for this tech giant.

## Main Analysis:
Unfortunately, specific news related to AAPL is currently unavailable. However, based on the previous uncertain trend prediction, it is essential for investors to stay informed and monitor the stock closely for any potential changes.

## Future Trend Prediction:
With the market in a volatile state and AAPL stock remaining sideways, it is crucial to keep a close eye on any upcoming developments. The fear/greed score being neutral signifies a balanced sentiment among investors. It is recommended to stay cautious and make informed decisions based on future news updates and price trends.

Remember to check reputable financial news websites for the most recent information on AAPL stock. Stay informed, stay cautious, and make wise investment decisions.